In [ ]:
import os

os.chdir("..")
print(f"Changed working directory to: {os.getcwd()}")

In [ ]:
import torch
import mlflow
import dagshub
import src.utils.data as data
import pytorch_lightning as pl

from torchvision import transforms
from src.models.ResNet50BinaryClassifier import ResNet50BinaryClassifier

mlflow.pytorch.autolog()
torch.set_float32_matmul_precision("high")

In [ ]:
# Erstellen Sie eine Instanz des Modells
model = ResNet50BinaryClassifier()
data_folder_path = "data/raw/burst_images/"

data_module = data.ECallistoDataModule(
    data_folder=data_folder_path,
    transform=transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((193, 240), antialias=True),
            transforms.ToTensor(),
        ]
    ),
    batch_size=64,
    num_workers=16,
    val_ratio=0.15,
    test_ratio=0.15,
    split_by_date=True,
    filter_instruments=["australia_assa_02"],
)
data_module.setup()

dagshub.init("FlareSense", "FlareSense", mlflow=True)
mlflow.start_run()
# Erstellen Sie einen Trainer für das Training
trainer = pl.Trainer(max_epochs=50)

# Starten Sie das Training
trainer.fit(
    model,
    train_dataloaders=data_module.train_dataloader(),
    val_dataloaders=data_module.val_dataloader(),
)

# Starten Sie die Tests
trainer.test(model, dataloaders=data_module.test_dataloader())

# Beenden Sie die MLflow-Sitzung
mlflow.end_run()